In [1]:
from bs4 import BeautifulSoup
import requests # sends requests to a website
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
website = 'https://www.teamrankings.com/ncaa-basketball/team/purdue-boilermakers/' # this is the website that I will parse
result = requests.get(website)
context = result.text
soup = BeautifulSoup(context, 'lxml')

In [3]:
 # list of teams to extract data from
teams = ['Purdue', 'Duke', 'Merrimack', 'Norfolk St', 'Oklahoma', 'New Mexico', 'Dayton', 'McNeese St', 'Baylor', 
         'UC Irvine', 'St Marys', 'Indiana St', 'Alabama', 'High Point', 'Wash State', 'Northwestern', 'Marquette', 
        'Morehead St', 'Arizona', 'S Dakota St', 'Texas', 'Boise St', 'Wisconsin', 'Richmond', 'Auburn', 'App State', 
        'BYU', 'Gonzaga', 'Creighton', 'Charl South', 'Florida', 'Nebraska', 'Kansas', 'E Washingtn', 'Connecticut', 'E Kentucky', 
        'Fla Atlantic', 'TX Christian', 'Texas Tech', 'Samford', 'San Diego St', 'Yale', 'Kentucky', 'Grd Canyon', 'Iowa St', 
         'Oakland', 'S Carolina', 'Nevada', 'N Carolina', 'Colgate', 'Houston', 'Sam Hous St', 'Utah St', 
         'Miss State', 'Clemson', 'Providence', 'Illinois', 'Akron', 'Colorado St', 'S Florida', 'Vermont', 
         'Michigan St', 'Virginia', 'Tennessee', 'Quinnipiac']
print(len(teams))
print(sorted(teams))
teams_urls = [] # list will eventually contain all urls to extract data from

65
['Akron', 'Alabama', 'App State', 'Arizona', 'Auburn', 'BYU', 'Baylor', 'Boise St', 'Charl South', 'Clemson', 'Colgate', 'Colorado St', 'Connecticut', 'Creighton', 'Dayton', 'Duke', 'E Kentucky', 'E Washingtn', 'Fla Atlantic', 'Florida', 'Gonzaga', 'Grd Canyon', 'High Point', 'Houston', 'Illinois', 'Indiana St', 'Iowa St', 'Kansas', 'Kentucky', 'Marquette', 'McNeese St', 'Merrimack', 'Michigan St', 'Miss State', 'Morehead St', 'N Carolina', 'Nebraska', 'Nevada', 'New Mexico', 'Norfolk St', 'Northwestern', 'Oakland', 'Oklahoma', 'Providence', 'Purdue', 'Quinnipiac', 'Richmond', 'S Carolina', 'S Dakota St', 'S Florida', 'Sam Hous St', 'Samford', 'San Diego St', 'St Marys', 'TX Christian', 'Tennessee', 'Texas', 'Texas Tech', 'UC Irvine', 'Utah St', 'Vermont', 'Virginia', 'Wash State', 'Wisconsin', 'Yale']


In [4]:
all_teams_dropdown = soup.find_all('select', class_ = 'redirectOnChange')[1] # finds dropdown menu containing all teams
options = all_teams_dropdown.find_all('option') #gets each team from the dropdown menu

teams_in_order = [] # gets teams in the same order that the urls are acquired so that team1 is populated correctly in later code

for option in options:
    if option.text in teams: # if each team exists in the teams array
        # get the end part of the url and store in teams_urls
        teams_in_order.append(option.text)
        url = option['value']
        teams_urls.append(url)
        
all_urls = ['https://www.teamrankings.com/' + url for url in teams_urls]    
print(len(all_urls))
print(len(teams_in_order))
print(teams_in_order)

65
65
['Akron', 'Alabama', 'App State', 'Arizona', 'Auburn', 'Baylor', 'Boise St', 'BYU', 'Charl South', 'Clemson', 'Colgate', 'Colorado St', 'Connecticut', 'Creighton', 'Dayton', 'Duke', 'E Kentucky', 'E Washingtn', 'Fla Atlantic', 'Florida', 'Gonzaga', 'Grd Canyon', 'High Point', 'Houston', 'Illinois', 'Indiana St', 'Iowa St', 'Kansas', 'Kentucky', 'Marquette', 'McNeese St', 'Merrimack', 'Michigan St', 'Miss State', 'Morehead St', 'N Carolina', 'Nebraska', 'Nevada', 'New Mexico', 'Norfolk St', 'Northwestern', 'Oakland', 'Oklahoma', 'Providence', 'Purdue', 'Quinnipiac', 'Richmond', 'S Carolina', 'S Dakota St', 'S Florida', 'Sam Hous St', 'Samford', 'San Diego St', 'St Marys', 'Tennessee', 'Texas', 'Texas Tech', 'TX Christian', 'UC Irvine', 'Utah St', 'Vermont', 'Virginia', 'Wash State', 'Wisconsin', 'Yale']


In [5]:
# initialize empty lists to be populated with data
team1 = []
team2 = []
team1_score = []
team2_score = []
location = []

In [6]:
index = 0 # index for keeping track of which team is being parsed, will be used for team1
for site in all_urls:
    # parse the given site
    result = requests.get(site)
    context = result.text
    soup = BeautifulSoup(context, 'lxml')
    
    # extract data
    
    # get the second table (the one containing game data)
    table = soup.find_all('table')[1]
    rows = table.find_all('tr') # get each row in the table
    del rows[0] #deletes the header

    for row in rows:
        columns = row.find_all('td') # get each column in each row
        team1.append(teams_in_order[index])

        # first column is opponent
        team2.append(columns[1].text)

        score = columns[2].text #score is in format "W/L score1-score2"
        split_dash = score.split('-')
        split_space = split_dash[0].split(' ')

        # if a game has not yet been played, add scores as None
        if len(split_dash) == 2:
            team1_score.append(int(split_space[1]))
            team2_score.append(int(split_dash[1]))
        else:
            team1_score.append(None)
            team2_score.append(None)

        location.append(columns[3].text)
        
    index += 1
    

In [7]:
df = pd.DataFrame({'team1':team1, 'team2':team2, 'team1_score':team1_score, 'team2_score':team2_score, 'team1_location':location})
print(df)

      team1          team2  team1_score  team2_score team1_location
0     Akron    S Dakota St         81.0         75.0           Away
1     Akron  S Mississippi         72.0         54.0           Home
2     Akron     Heidelberg        114.0         56.0           Home
3     Akron   Florida Intl         77.0         71.0        Neutral
4     Akron        Utah St         62.0         65.0        Neutral
...     ...            ...          ...          ...            ...
2005   Yale        Cornell         62.0         65.0           Away
2006   Yale       Columbia         84.0         76.0           Away
2007   Yale      Dartmouth         80.0         56.0           Home
2008   Yale        Harvard         80.0         60.0           Home
2009   Yale          Brown          NaN          NaN           Home

[2010 rows x 5 columns]


In [8]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import sklearn.datasets
from pandas import DataFrame

In [9]:
# import team data
data_dump = pd.read_csv('march_madness_data_dump.csv')
data_dump.head()

,Rk,School,Win-Loss Percentage,SRS,SOS,Team Points,Opponent Ponts,Minutes Played,FG,FGA,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,1,Abl Christian,0.375,-5.82,-1.21,1756,1811,975,614,1433,...,402,558,0.720,236,811,304,190,47,315,493
1,2,Air Force,0.333,-3.73,-0.61,1630,1705,970,571,1248,...,272,403,0.675,188,708,363,165,96,299,413
2,3,Akron,0.750,5.49,-1.42,1804,1551,965,631,1378,...,337,457,0.737,254,883,324,150,68,270,389
3,4,Alabama,0.708,23.98,10.57,2167,1845,960,726,1512,...,437,550,0.795,300,948,381,183,101,295,459
4,5,Alab A&M,0.250,-16.23,-5.23,1656,1920,965,550,1332,...,453,635,0.713,276,848,237,183,88,402,520


In [10]:
# merge the team data with the game data, with the school name being the common column
merge1 = pd.merge(df, data_dump, left_on = 'team1', right_on = 'School', how = 'left')
final = pd.merge(merge1, data_dump, left_on = 'team2', right_on = 'School', how = 'left', suffixes = ('_team1', '_team2'))

# find all rows with null values, either from unplayed games or missing info in the data_dump
bad_rows = final[final.isnull().any(axis=1)]
print(bad_rows[['team1', 'team2', 'team1_score', 'team2_score']])

# remove all rows with null values from final
final = final[~final.isin(bad_rows)].dropna()

# drop unnecessary columns
final = final.drop(columns = ['team1', 'team2', 'Rk_team1', 'Rk_team2', 'School_team1', 'School_team2'])

# final output visualization
print(final.head())
print(final.shape)

          team1           team2  team1_score  team2_score
2         Akron      Heidelberg        114.0         56.0
9         Akron  Miami-Hamilton         88.0         45.0
29        Akron      E Michigan          NaN          NaN
30        Akron      W Michigan          NaN          NaN
60      Alabama         Florida          NaN          NaN
...         ...             ...          ...          ...
1979  Wisconsin         Rutgers          NaN          NaN
1980  Wisconsin          Purdue          NaN          NaN
1981       Yale  Vassar College        102.0         53.0
1991       Yale    Colby-Sawyer         95.0         36.0
2009       Yale           Brown          NaN          NaN

[242 rows x 4 columns]
   team1_score  team2_score team1_location  Win-Loss Percentage_team1  \
0         81.0         75.0           Away                       0.75   
1         72.0         54.0           Home                       0.75   
3         77.0         71.0        Neutral                   

In [13]:
# set the target to the score margin (score1 - score2)
target = final['team1_score'] - final['team2_score']

# drop score columns, as they make up the margin
X = final.drop(columns = ['team1_score', 'team2_score'])
X['margin'] = target
# perform get_dummies on the location column
X = pd.get_dummies(X, columns = ['team1_location'])
print(X.shape)


(1768, 48)


In [14]:
X.to_csv('MarchMadnessData2024.csv', index=False)
